In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Bidirectional, Input, Embedding, Activation, GRU
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import backend as K
import pickle
import numpy as np
from model_CONV import dp_syllabler

In [2]:
#ortho max encoder len = 45
#ipa max encoder len = 74

with open('data/e2i_ortho.pkl', 'rb') as file:
    e2i_ortho = pickle.load(file)
    
with open('data/e2i_ipa.pkl', 'rb') as file:
    e2i_ipa = pickle.load(file)

shitter = dp_syllabler(e2i_ortho= e2i_ortho, e2i_ipa = e2i_ipa, ortho_input_size=45,ipa_input_size=74,latent_dim=50,embed_dim=50 ,max_feat=148)

In [3]:
def to_categorical(sequences):
        cat_sequences = []
        for s in sequences:
            cats = []
            for item in s:
                cats.append(np.zeros(3))
                cats[-1][item] = 1.0
            cat_sequences.append(cats)
        return np.array(cat_sequences)

In [4]:
with open('data/x_tr_ortho.pkl', 'rb') as file:
    x_tr_ortho = pickle.load(file)
    # x_tr_ortho = x_tr_ortho.reshape(len(x_tr_ortho), 45, 1)
    
with open('data/x_tr_ipa.pkl', 'rb') as file:
    x_tr_ipa = pickle.load(file)
    # x_tr_ipa = x_tr_ipa.reshape(len(x_tr_ipa), 74, 1)
    
with open('data/y_tr.pkl', 'rb') as file:
    y_tr = pickle.load(file)
    y_tr = y_tr.reshape(len(y_tr), 74, 1)
    
y_tr = to_categorical(y_tr)
split_index = int(.8 * len(x_tr_ortho))

x_test_ortho = x_tr_ortho[split_index:]
x_test_ipa = x_tr_ipa[split_index:]
x_test_ipa = x_test_ipa.reshape(len(x_test_ipa), 74, 1)
y_test = y_tr[split_index:]

x_tr_ortho = x_tr_ortho[:split_index]
x_tr_ipa = x_tr_ipa[:split_index]
y_tr = y_tr[:split_index]

In [6]:
shitter.fit(x_tr_ortho, x_tr_ipa, y_tr, x_test_ortho, x_test_ipa, y_test, ep=50, batch_size=128, save_filename='data/double_pen_best_weights.h5')

Epoch 1/50
63/63 [==============================] - 115s 1s/step - loss: 0.2265 - accuracy: 0.9288 - ignore_accuracy: 0.4860 - val_loss: 0.0585 - val_accuracy: 0.9765 - val_ignore_accuracy: 0.7752

Epoch 00001: accuracy improved from -inf to 0.92882, saving model to data\double_pen_best_weights.h5
Epoch 2/50
63/63 [==============================] - 72s 1s/step - loss: 0.0474 - accuracy: 0.9783 - ignore_accuracy: 0.7853 - val_loss: 0.0439 - val_accuracy: 0.9780 - val_ignore_accuracy: 0.7856

Epoch 00002: accuracy improved from 0.92882 to 0.97828, saving model to data\double_pen_best_weights.h5
Epoch 3/50
63/63 [==============================] - 71s 1s/step - loss: 0.0419 - accuracy: 0.9785 - ignore_accuracy: 0.7874 - val_loss: 0.0417 - val_accuracy: 0.9780 - val_ignore_accuracy: 0.7855

Epoch 00003: accuracy improved from 0.97828 to 0.97850, saving model to data\double_pen_best_weights.h5
Epoch 4/50
63/63 [==============================] - 71s 1s/step - loss: 0.0396 - accuracy: 0.9790 -

In [ ]:
def back_to_eng(arr):
    worded = []
    for i in arr:
        if i != 0:
            worded += list(e2i_ortho.keys())[list(e2i_ortho.values()).index(i)]
    return worded

def back_to_ipa(arr):
    worded = []
    for i in arr:
        if i != 0:
            worded += list(e2i_ipa.keys())[list(e2i_ipa.values()).index(i)]
    return worded

In [ ]:
shitter.model.load_weights('data/double_pen_no_dense_best_weights.h5')

In [ ]:
test_ortho = ''.join(back_to_eng(x_test_ortho[7]))
test_ipa = ''.join(back_to_ipa(x_test_ipa[7]))

In [11]:
print(shitter.syllabify('finkle', '/fɪnkʌl/'))

fin-kle


In [ ]:
'frumdafinkle', 'fɹʌmdʌfʌŋkʌl'